In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np

df=pd.read_csv('/kaggle/input/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
y=df['Survived']
df.drop(['Survived'], axis=1, inplace=True)

In [6]:
df.drop(['Name', 'PassengerId', 'Ticket'], axis=1, inplace=True)

In [7]:
df['Sex']=[0 if i == 'male' else 1 for i in df.Sex ]

In [8]:
df['Age'].fillna(df.Age.mean(), inplace=True)

In [9]:
df['Cabin'].fillna(0, inplace=True)
df['Cabin']=[0 if i== 0 else 1 for i in df.Cabin]
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,0,22.0,1,0,7.2500,0,S
1,1,1,38.0,1,0,71.2833,1,C
2,3,1,26.0,0,0,7.9250,0,S
3,1,1,35.0,1,0,53.1000,1,S
4,3,0,35.0,0,0,8.0500,0,S


In [10]:
df.drop('Embarked', axis=1, inplace=True)

In [11]:
age=df[['Age']].values
age=age.reshape(-1, 1)

In [12]:
fare=df[['Fare']].values
fare=fare.reshape(-1, 1)

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df['Age']=scaler.fit_transform(age)
df['Fare']=scaler.fit_transform(fare)

In [14]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin
0,3,0,0.271174,1,0,0.014151,0
1,1,1,0.472229,1,0,0.139136,1
2,3,1,0.321438,0,0,0.015469,0
3,1,1,0.434531,1,0,0.103644,1
4,3,0,0.434531,0,0,0.015713,0


In [15]:
pclass=df[['Pclass']].values
pclass=pclass.reshape(-1, 1)

In [16]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(categories='auto')
data_pclass=ohe.fit_transform(pclass).toarray()

In [17]:
data=data_pclass[:,1:]
data

array([[0., 1.],
       [0., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 0.],
       [0., 1.]])

In [18]:
df1=pd.DataFrame(data, columns=['pclass1', 'pclass2'])
df=pd.concat([df, df1], axis=1)
df.drop('Pclass', axis=1, inplace=True)

In [19]:
df.head()

,Sex,Age,SibSp,Parch,Fare,Cabin,pclass1,pclass2
0,0,0.271174,1,0,0.014151,0,0.0,1.0
1,1,0.472229,1,0,0.139136,1,0.0,0.0
2,1,0.321438,0,0,0.015469,0,0.0,1.0
3,1,0.434531,1,0,0.103644,1,0.0,0.0
4,0,0.434531,0,0,0.015713,0,0.0,1.0


In [20]:
df.isnull().sum()

Sex        0
Age        0
SibSp      0
Parch      0
Fare       0
Cabin      0
pclass1    0
pclass2    0
dtype: int64

In [21]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
X_train, X_test, y_train, y_test= train_test_split(df, y, test_size=0.2)

In [22]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=150, random_state=0)
param_dist = {"max_depth": [8, 10, 12],
              "max_features": [8, 8, 8],
              "min_samples_split": [3, 4, 5],
              'min_samples_leaf': [2, 3, 4],
              "bootstrap": [True, True, True],
              "criterion": ['entropy', "gini", "gini"]}

n_iter_search = 25
random_search = RandomizedSearchCV(rfc, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=150,
                                                    n_jobs=None,
 

In [23]:
y_pred=random_search.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(y_test, y_pred))

0.9050279329608939


In [25]:
confusion_matrix(y_test, y_pred)

array([[101,   9],
       [  8,  61]])

In [52]:
test_df=pd.read_csv('/kaggle/input/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [53]:
def clean_dataset(df):
    passengerId=df['PassengerId']
    df.drop(['Name', 'PassengerId', 'Ticket', 'Embarked'], axis=1, inplace=True)
    df['Sex']=[0 if i == 'male' else 1 for i in df.Sex]
    df['Age'].fillna(df.Age.mean(), inplace=True)
    df['Cabin'].fillna(0, inplace=True)
    df['Fare'].fillna(df.Fare.mean(), inplace=True)
    df['Cabin']=[0 if i== 0 else 1 for i in df.Cabin]
    age=df[['Age']].values
    age=age.reshape(-1, 1)
    fare=df[['Fare']].values
    fare=fare.reshape(-1, 1)
    df['Age']=scaler.fit_transform(age)
    df['Fare']=scaler.fit_transform(fare)
    pclass=df[['Pclass']].values
    pclass=pclass.reshape(-1, 1)
    data_pclass=ohe.fit_transform(pclass).toarray()
    data=data_pclass[:,1:]
    df1=pd.DataFrame(data, columns=['pclass1', 'pclass2'])
    df=pd.concat([df, df1], axis=1)
    df.drop('Pclass', axis=1, inplace=True)
    return df

In [54]:
test_df1= clean_dataset(test_df)

In [55]:
test_df1.head()

,Sex,Age,SibSp,Parch,Fare,Cabin,pclass1,pclass2
0,0,0.452723,0,0,0.015282,0,0.0,1.0
1,1,0.617566,1,0,0.013663,0,0.0,1.0
2,0,0.815377,0,0,0.018909,0,1.0,0.0
3,0,0.353818,0,0,0.016908,0,0.0,1.0
4,1,0.287881,1,1,0.023984,0,0.0,1.0


In [56]:
test_df1.isnull().sum()

Sex        0
Age        0
SibSp      0
Parch      0
Fare       0
Cabin      0
pclass1    0
pclass2    0
dtype: int64

In [57]:
pred_test_dataset=random_search.predict(test_df1)

In [58]:
pred_test_dataset

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [59]:
pred_test_dataset_df=pd.DataFrame(pred_test_dataset, columns=['Survived'])

In [64]:
df_test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [106]:
pd.concat([df_test['PassengerId'], pred_test_dataset_df['Survived']], axis=1).to_csv(('/kaggle/input/submission_file.csv'), index=None, header=True)

In [98]:
! cd /kaggle/working 

In [99]:
! ls -la

total 20
drwxr-xr-x 3 root root 4096 Oct  7 10:41 .
drwxr-xr-x 6 root root 4096 Oct  7 09:59 ..
drwxr-xr-x 2 root root 4096 Oct  7 09:59 .ipynb_checkpoints
-rw-r--r-- 1 root root  199 Oct  7 09:59 __notebook_source__.ipynb
-rw-r--r-- 1 root root 2839 Oct  7 10:46 submission_file.csv


In [108]:
from IPython.display import FileLink
FileLink(r'submission_file.csv')

/kaggle/working/submission_file.csv